In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import time
import torch
from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig, GSTConfig
from TTS.tts.datasets import load_tts_samples
from TTS.utils.audio import AudioProcessor
from TTS.trainer_windows import Trainer, TrainingArgs

# Old Tacotron Imports
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.configs.shared_configs import GSTConfig

# Style Tacotron Imports
from TTS.tts.models.styletacotron2 import Styletacotron2
from TTS.tts.configs.styletacotron2_config import Styletacotron2Config
from TTS.style_encoder.configs.style_encoder_config import StyleEncoderConfig

# Style forward TTS Imports
from TTS.tts.models.styleforward_tts import StyleForwardTTS
from TTS.tts.configs.stylefast_pitch_config import StyleFastPitchConfig

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\torchaudio\backend\utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to 

In [3]:
seed_everything(42)
output_path = './'

# init configs
dataset_config = BaseDatasetConfig(
    name="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "D:/Mestrado/Emotion Audio Synthesis (TTS)/repo_final/pt_etts/data/LJSpeech\LJSpeech-1.1")
)

audio_config = BaseAudioConfig(
    sample_rate=22050,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

style_config = StyleEncoderConfig(se_type="vaeflow",
                                 use_proj_linear = True,
                                 use_nonlinear_proj = True,
                                 proj_dim = 384,
                                 agg_type = 'sum',
                                 start_loss_at = 500)

config = StyleFastPitchConfig(  # This is the config that is saved for the future use
    style_encoder_config = style_config,
    audio=audio_config,
    batch_size=64,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    r=6,
#     gradual_training=[[0, 6, 64], [10000, 4, 32], [50000, 3, 32], [100000, 2, 32]],
#     double_decoder_consistency=True,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=True,
    mixed_precision=False,
    output_path=output_path,
    datasets=[dataset_config],
)
# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True)

# init model
model = StyleForwardTTS(config)

# # init the trainer and 🚀
# trainer = Trainer(
#     TrainingArgs(),
#     config,
#     output_path,
#     model=model,
#     train_samples=train_samples,
#     eval_samples=eval_samples,
#     training_assets={"audio_processor": ap},
# )
# # Data loader
# trainer.train_loader = trainer.get_train_dataloader(trainer.training_assets,trainer.train_samples, verbose=True)# # init the trainer and 🚀
# trainer = Trainer(
#     TrainingArgs(),
#     config,
#     output_path,
#     model=model,
#     train_samples=train_samples,
#     eval_samples=eval_samples,
#     training_assets={"audio_processor": ap},
# )
# # Data loader
# trainer.train_loader = trainer.get_train_dataloader(trainer.training_assets,trainer.train_samples, verbose=True)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 | > Found 13100 files in D:\Mestrado\Emotion Audio Synthesis (TTS)\repo_final\pt_etts\data\LJSpeech\LJSpeech-1.1


In [4]:
model

StyleForwardTTS(
  (emb): Embedding(130, 384)
  (encoder): Encoder(
    (encoder): FFTransformerBlock(
      (fft_layers): ModuleList(
        (0): FFTransformer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=384, out_features=384, bias=True)
          )
          (conv1): Conv1d(384, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1024, 384, kernel_size=(3,), stride=(1,), padding=(1,))
          (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): FFTransformer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=384, out_features=384, bias=True)
          )
          (conv1): Conv1d(384, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Co

In [7]:
model.get_criterion()

VAEFlow report - Using Cyclical Annealing:  True


StyleForwardTTSLoss(
  (spec_loss): MSELossMasked()
  (dur_loss): MSELossMasked()
  (aligner_loss): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=3)
    (ctc_loss): CTCLoss()
  )
  (pitch_loss): MSELossMasked()
  (ssim): SSIMLoss()
  (criterion_se): VAEFlowStyleEncoderLoss()
)

In [19]:
# example forward

from torch import nn
import torch

y = torch.rand((16,80))
y_lengths = torch.rand((16)).type(torch.LongTensor)
x = torch.rand((16, 20)).type(torch.LongTensor)
x_lengths = torch.rand((16,20)).type(torch.LongTensor)
dr = torch.rand((16,20)).type(torch.IntTensor)
pitch = torch.rand((16,1,30))

# y_lengths.shape
model.cpu().forward(x,x_lengths, y_lengths, y, dr = dr, pitch = pitch)

AssertionError: 

In [20]:
# init the trainer and 🚀
trainer = Trainer(
    TrainingArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)
# Data loader
trainer.train_loader = trainer.get_train_dataloader(trainer.training_assets,trainer.train_samples, verbose=True)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [ ]:
batch